## Thumbnail Creation
Create a circular thumbnail for each movie based on an input image. 

In [16]:
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
from PIL import Image, ImageDraw


def draw_ellipse(image, bounds, width=1, outline='white', antialias=4):
    """Improved ellipse drawing function, based on PIL.ImageDraw."""

    # Use a single channel image (mode='L') as mask.
    # The size of the mask can be increased relative to the imput image
    # to get smoother looking results. 
    mask = Image.new(
        size=[int(dim * antialias) for dim in image.size],
        mode='L', color='black')
    draw = ImageDraw.Draw(mask)

    # draw outer shape in white (color) and inner shape in black (transparent)
    for offset, fill in (width/-2.0, 'white'), (width/2.0, 'black'):
        left, top = [(value + offset) * antialias for value in bounds[:2]]
        right, bottom = [(value - offset) * antialias for value in bounds[2:]]
        draw.ellipse([left, top, right, bottom], fill=fill)

    # downsample the mask using PIL.Image.LANCZOS 
    # (a high-quality downsampling filter).
    mask = mask.resize(image.size, Image.LANCZOS)
    # paste outline color to input image through the mask
    image.paste(outline, mask=mask)


def transform_image(title, path, save_path):
    # Open the input image as numpy array, convert to RGB
    img=Image.open(f"{path}{title}.jpg").convert("RGB")
    h,w=img.size
    
    if h != w:
        img = img.resize((h, h))
        
    h,w=img.size
    
    npImage=np.array(img)
    

    # Create same size alpha layer with circle
    alpha = Image.new('L', img.size,0)
    draw = ImageDraw.Draw(alpha)
    draw.pieslice([0,0,h,w],0,360,fill=255)

    # Convert alpha Image to numpy array
    npAlpha=np.array(alpha)

    # Add alpha layer to RGB
    npImage=np.dstack((npImage,npAlpha))


    result = Image.fromarray(npImage)
        
    # Add circle
    width = int(h * 0.04)
    half_width = int(width/2)
    ellipse_box = [half_width, half_width, h-half_width, w-half_width]
    draw_ellipse(result, ellipse_box, width=width, outline='black')
    
    
    
#     ellipse_box = [10, 10, h-10, w-10]
#     draw_ellipse(result, ellipse_box, width=20, outline='black')
    
    result.save(f"{save_path}{title}.png")

In [17]:
pixar = pd.read_csv("pixar_with_seed.csv")
disney = pd.read_csv("disney_with_seed.csv")

### Pixar

In [41]:
titles = list(pixar.Title.values)
titles = [title.lower().replace("·", " ").replace(",", " ").replace(".", " ").replace("'", " ").strip() for title in titles]
titles = [title.replace("  ", " ").replace("& ", "") for title in titles]

In [42]:
for title in titles:
    image = transform_image(title, path="images/pixar/", save_path="images/pixar/small/")

### Disney

In [32]:
titles = list(disney.Title.values)
titles = [title.lower().replace("·", " ").replace(",", " ").replace(".", " ").replace("'", " ").strip() for title in titles]
titles = [title.replace("  ", " ").replace("& ", "").replace(":", "").replace("-", " ") for title in titles]

In [34]:
for title in titles:
    image = transform_image(title, path="images/disney/", save_path="images/disney/small/")